In [24]:
import pandas as pd
import numpy as np
import os
import ast

#explode=False
#all_registers=False


In [26]:
datas = []

labels_all_hierarchy_with_other = {
    "MT": ["MT"],
    "LY": ["LY"],
    "SP": ["SP", "it", "os"],
    "ID": ["ID"],
    "NA": ["NA", "ne", "sr", "nb", "on"],
    "HI": ["HI", "re", "oh"],
    "IN": ["IN", "en", "ra", "dtp", "fi", "lt", "oi"],
    "OP": ["OP", "rv", "ob", "rs", "av", "oo"],
    "IP": ["IP", "ds", "ed", "oe"],
}

final_scores = {"MT": 0.374349,
    "LY": 0.357626,
    "SP": 0.422314,
    "ID": 0.430686,
    "NA": 0.441469,
    "HI": 0.446558,
    "IN": 0.437347,
    "OP":  0.446629,
    "IP": 0.393026,
    "HI-IN":0.464502,
    "ne": 0.417933,
    "dtp": 0.452347,
    }

with os.scandir("result") as f:
    for file in f:
        filename = file.name
        print(f"reading {filename}")
        with open("result/"+filename) as f:
            data = [ast.literal_eval(line) for line in f]
        df_ = pd.DataFrame(data)
        df_ = df_.drop(columns="text")
        df_["parsed_register"] = df_["register"].map(lambda reg: [r for r in reg if r in list(labels_all_hierarchy_with_other.keys())+["ne", "dtp"]])
        df_["parsed_register"] = df_["parsed_register"].map(lambda reg: "-".join(reg))
        df_["parsed_register"] = df_["parsed_register"].map(lambda reg: {"NA-ne":"ne", "ne-NA": "ne", "IN-dtp":"dtp", "dtp-IN":"dtp", "IN-HI":"HI-IN"}.get(reg, reg))
        df_["final_score"] = df_["parsed_register"].map(lambda reg: final_scores[reg])
        df_["dclm-label"] = df_["dclm-label"].map(lambda sc: {"__label__cc": 0, "__label__hq": 1}[sc])
        datas.append(df_)



df = pd.concat(datas)

reading MT_with_edu_and_dclm.jsonl
reading NA_with_edu_and_dclm.jsonl
reading OP_with_edu_and_dclm.jsonl
reading HI_with_edu_and_dclm.jsonl
reading IP_with_edu_and_dclm.jsonl
reading LY_with_edu_and_dclm.jsonl
reading ID_with_edu_and_dclm.jsonl
reading HI-IN_with_edu_and_dclm.jsonl
reading dtp_with_edu_and_dclm.jsonl
reading ne_with_edu_and_dclm.jsonl
reading IN_with_edu_and_dclm.jsonl
reading SP_with_edu_and_dclm.jsonl


In [27]:
df.rename(columns={"dclm-label": "dclmlabel"}, inplace=True)
display(df)
print(np.unique(df["parsed_register"].tolist(), return_counts=True))

,id,register,score,int_score,dclmlabel,dclm-prob,parsed_register,final_score
0,aa6e226e50524b13656a48cca50ef88b,[MT],0.226472,0,0,0.999351,MT,0.374349
1,48ea34d9aa0ce1a6ac031ee3ee950eda,[MT],0.365965,0,0,0.999481,MT,0.374349
2,7d8fda8eca0b17ff6faab5f5837356dd,[MT],0.550137,1,0,0.999869,MT,0.374349
3,820661e03daf8d1268772cd7a0c84856,[MT],-0.114357,0,0,0.997457,MT,0.374349
4,56831efa9c0dbe62578889db0c02cd2d,[MT],-0.280572,0,0,1.000010,MT,0.374349
...,...,...,...,...,...,...,...,...
9995,b318fb00ae2ebbdc7b967d351d85cb25,"[it, SP]",1.360814,1,0,0.996769,SP,0.422314
9996,530ee8b4002b53361b8277d5a81a9033,"[SP, it]",0.841825,1,0,0.959559,SP,0.422314
9997,12b07b501cef77656fa8df40c38223eb,"[it, SP]",0.671556,1,0,0.931684,SP,0.422314
9998,7ace17c3e00e7d8ca4f2c27a72ce1760,"[SP, it]",1.580386,2,0,0.987279,SP,0.422314


(array(['HI', 'HI-IN', 'ID', 'IN', 'IP', 'LY', 'MT', 'NA', 'OP', 'SP',
       'dtp', 'ne'], dtype='<U5'), array([10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000,
       10000, 10000, 10000]))


In [28]:
def cooccurrence_matrix(df):
    regs = np.unique(df["parsed_register"]).tolist()
    gens = np.unique(df["dclmlabel"]).tolist()
    print("regs: ", regs)
    print("gens: ", gens)

    print(range(len(regs)))
    reg2index = {k:v for k,v in zip(regs, range(len(regs)))}
    gen2index = {k:v for k,v in zip(gens, range(len(gens)))}

    m = np.zeros((len(regs), len(gens)), dtype=np.float32)
    summa = 0
    for i,d in df.iterrows():
        r = d["parsed_register"]
        g = d["dclmlabel"]

        m[reg2index[r],gen2index[g]] += 1
        summa+=1

    return m/summa


m = cooccurrence_matrix(df)

assert 0.999 < sum(m.sum(axis=0)) < 1.001 and 0.999 < sum(m.sum(axis=1)) < 1.001

regs:  ['HI', 'HI-IN', 'ID', 'IN', 'IP', 'LY', 'MT', 'NA', 'OP', 'SP', 'dtp', 'ne']
gens:  [0, 1]
range(0, 12)


In [29]:
print(sum(m.sum(axis=0)))
print(sum(m.sum(axis=1)))

0.9999999999999999
1.0


In [30]:
def entropy(m):
    p_gen = m.sum(axis=0)
    p_reg = m.sum(axis=1)

    def _entropy_1d(v):
        return -1*sum(v*np.log2(v))

    def _entropy_2d(m):
        ent = 0
        for j in m:
            for k in j:
                if k != 0:
                    ent += k*np.log2(k)
        return -1*ent

    ent_reg = _entropy_1d(p_reg)
    ent_gen = _entropy_1d(p_gen)
    ent_reg_gen = _entropy_2d(m)

    return ent_reg, ent_gen, ent_reg_gen


def mutual_info(m):
    reg_pdf = m.sum(axis=1)
    gen_pdf = m.sum(axis=0)

    info = 0
    for i in range(len(reg_pdf)):
        for j in range(len(gen_pdf)):
            if m[i,j]!= 0:
                info += m[i,j]*np.log(m[i,j]/(reg_pdf[i]*gen_pdf[j]))
    return info


print(entropy(m))
print(mutual_info(m))

(3.584962500721156, 0.20297021758853812, 3.773747887152132)
0.009832175723583594


In [31]:
from scipy.stats import chi2_contingency

def cramers_v(df):
    x = df["parsed_register"]
    y = df["dclmlabel"]

    confusion_matrix = pd.crosstab(x, y)
    print(confusion_matrix)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))
print(cramers_v(df))

dclmlabel           0    1
parsed_register           
HI               9814  186
HI-IN            9487  513
ID               9266  734
IN               9564  436
IP               9941   59
LY               9180  820
MT               9864  136
NA               9923   77
OP               9780  220
SP               9625  375
dtp              9810  190
ne               9939   61
0.14212750174297809


In [32]:
! pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [33]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Assuming 'nominal_var' is your nominal column and 'ordinal_var' is numeric (1 to 5)
model = ols('dclmlabel ~ C(parsed_register)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
eta_squared = anova_table['sum_sq']['C(parsed_register)'] / anova_table['sum_sq'].sum()
print(eta_squared)

0.020291725845637414


In [34]:
! pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [35]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Encode feature1
df['feature1_encoded'] = le.fit_transform(df['parsed_register'])

# Encode feature2
df['feature2_encoded'] = le.fit_transform(df['dclmlabel'])

# Extract the encoded features
X = df[['feature1_encoded']]
y = df['feature2_encoded']

# Calculate mutual information
mi = mutual_info_classif(X, y, discrete_features=True)

print(f'Mutual Information between feature1 and feature2: {mi[0]}')

Mutual Information between feature1 and feature2: 0.009832175723584274


In [36]:
! pip install plotly
! pip install nbformat>=4.2.0

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [37]:
import pandas as pd
import plotly.graph_objects as go

# Example: sample data
# df = pd.DataFrame({
#     'parsed_register': ['A', 'B', 'A', 'C', 'B', 'C', 'A', 'B', 'C', 'A'],
#     'int_score': [1, 2, 2, 3, 3, 4, 1, 5, 4, 2]
# })

# Count pairs (grouped by nominal and ordinal labels)
df['dclmlabel'] = df['dclmlabel'].astype(str)  # make sure it's a string for plotting
counts = df.groupby(['parsed_register', 'dclmlabel']).size().reset_index(name='count')

# Get all unique labels
all_labels = list(pd.unique(counts['parsed_register'].tolist() + counts['dclmlabel'].tolist()))
label_to_index = {label: i for i, label in enumerate(all_labels)}

# Build Sankey inputs
source_indices = counts['parsed_register'].map(label_to_index)
target_indices = counts['dclmlabel'].map(label_to_index)
values = counts['count']

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=all_labels,
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values
    )
)])

fig.update_layout(title_text="Sankey Diagram: Register vs DCLM score", font_size=12, width=600)
fig.show()
